<a href="https://colab.research.google.com/github/juyoung524/22-1-Spring-/blob/main/20220530(tensorflow_hub).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [Tensorflow Hub](https://www.tensorflow.org/hub)

### image classification (e.g. [inception_v3](https://tfhub.dev/google/imagenet/inception_v3/classification/5))

image classification의 예를 밑에 구현해둔 것임.

In [1]:
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub

필요한 library 가져오기

In [2]:
model = hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/5")

model을 이런 방식으로 가져옴. 이 모델 자체가 함수같이 쓰임 (pipeline때와 비슷)

In [3]:
from PIL import Image
import requests
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

고양이 그림을 url로 지정, 받아옴 (PIL이라는 형태- 아직 numerically 쓸 수 없는 형태)

In [4]:
import numpy as np
image = np.array(image)
# reshape into shape [batch_size, height, width, num_channels]
image = tf.reshape(image, [1, image.shape[0], image.shape[1], image.shape[2]])
# Use `convert_image_dtype` to convert to floats in the [0,1] range.
image = tf.image.convert_image_dtype(image, tf.float32)

그림을 numpy 형태로 array 만들어줘야 함. 여기에 reshape해서 예전에 numpy reshape해줬던 것처럼, 3차원 -> 4차원의 array로 바꿔줘야 함

(Model 자체가 4차원으로 받기 때문임)

한 번 더 convert: 쓰려고 하는 model 자체는 0에서 1 사이로 (어두우면 0, 완전 밝은 색이면 1 이런 식으로 그 사이의 숫자로 되어 있음) **0~1 사이의 숫자로 받아줘야 함**

image[:,:,0] 해서 보면 0부터 255까지 256개로 되어 있음

한꺼번에 처리하기 위해 4차원으로 처리하도록 함

In [7]:
logits = model(image)
logits

<tf.Tensor: shape=(1, 1001), dtype=float32, numpy=
array([[ 0.13278216, -0.98323435,  0.4061772 , ..., -0.12934873,
         0.38595918,  0.4459334 ]], dtype=float32)>

이런 값들이 들어와 있음. 어떤 게 제일 높은 값인가? (1000개 중에서) - argmax라는 함수

In [8]:
np.argmax(logits)

282

282번째가 가장 높은 값임

사용된 label이 1000개가 있음. 282번째 것이 가장 확률이 높더라. model에서 사용되던 label임.

다 copy해서 프로그램 짜도 되지만, 엑셀에 copy해서 볼 것임. 밑으로 쭉 내려서 282번째가 뭔지 볼 수 있음. 283 (0부터니까)번째: Tabby cat (제대로 인식해냄)

model 안에 행렬들이 한참 들어 있음.

> https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt